# Usando un modelo de Inteligencia Artificial previamente entrenado

En este Notebook simularemos ser un **ambiente de producción** (ej: un servidor web o la computadora de un amigo). Ya no necesitamos los datos de entrenamiento ni entrenar nada. Solo necesitamos:

1. El **Cerebro** (`spam_model.pth`)
2. El **Traductor** (`vectorizer.pkl`)
3. La **Estructura** del modelo (La clase `SpamDetector`)


In [ ]:
import torch
import torch.nn as nn
import joblib

## 1. Definir la Estructura

**IMPORTANTE:** Para cargar un modelo de PyTorch, necesitamos tener definida la clase `SpamDetector` exactamente igual a como estaba cuando lo entrenamos. Si no, PyTorch no sabrá dónde poner los pesos.

In [ ]:
# Esta configuración debe ser IGUAL a la original
MAX_PALABRAS = 100 

class SpamDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.capa_oculta = nn.Linear(MAX_PALABRAS, 20)
        self.activacion = nn.ReLU()
        self.capa_salida = nn.Linear(20, 2)
    
    def forward(self, x):
        x = self.capa_oculta(x)
        x = self.activacion(x)
        x = self.capa_salida(x)
        return x

print("Estructura del modelo definida.")

## 2. Cargar el Cerebro y el Traductor

Vamos a revivir a nuestra IA cargando los archivos guardados.

In [ ]:
# 1. Cargar el Traductor
try:
    loaded_vectorizer = joblib.load('vectorizer.pkl')
    print("Traductor cargado exitosamente")
except FileNotFoundError:
    print("ERROR: No se encuentra 'vectorizer.pkl'. Asegúrate de haberlo guardado en el otro notebook.")

# 2. Cargar el Modelo
try:
    # Inicializamos una modelo vacio
    loaded_model = SpamDetector()
    
    # Le cargamos los pesos guardados
    loaded_model.load_state_dict(torch.load('spam_model.pth'))
    
    # La ponemos en modo evaluación
    loaded_model.eval() 
    print("Modelo cargado exitosamente")
except FileNotFoundError:
    print("ERROR: No se encuentra 'spam_model.pth'. Asegúrate de haberlo guardado en el otro notebook.")

## 3. ¡A Predecir!

Ahora podemos usar esta función para detectar Spam en nuevos correos.

In [ ]:
def predecir_nuevo_correo(texto):
    # 1. Usar el vectorizer cargado
    x_num = loaded_vectorizer.transform([texto]).toarray()
    x_tensor = torch.tensor(x_num, dtype=torch.float32)
    
    # 2. Usar el modelo cargado
    with torch.no_grad():
        prediccion = loaded_model(x_tensor)
        prob = torch.softmax(prediccion, dim=1)
        
        clase = torch.argmax(prob).item()
        conf = prob[0][clase].item() * 100
        
    etiqueta = "SPAM" if clase == 1 else "LEGÍTIMO"
    return f"{etiqueta} (Confianza: {conf:.1f}%)"

# Testeo
print(f"Correo: 'Gana dinero ya' -> {predecir_nuevo_correo('Gana dinero ya')}")
print(f"Correo: 'Reunion mañana' -> {predecir_nuevo_correo('Reunion mañana')}")

In [ ]:
# ¡Prueba con tus propios mensajes!
mensaje = "Oferta exclusiva solo por hoy"
print(f"Analizando: '{mensaje}'")
print(predecir_nuevo_correo(mensaje))